# 7. Understanding ROS Services and Parameters
http://wiki.ros.org/ROS/Tutorials/UnderstandingServicesParams

In [1]:
import os
import sys

home_dir = os.getcwd()
home_dir

'/home/pi/notebooks/Tutorials/1_Core_ROS_Tutorials'

In [2]:
# get ROS related environment variables

envs = !printenv | grep ROS 
ROS_ENVs = {}

for env in envs:
    env = env.split('=')
    [k, v] = env
    ROS_ENVs[k] = v

ROS_ENVs['LD_LIBRARY_PATH'] = '/opt/ros/kinetic/lib'
ROS_ENVs['PKG_CONFIG_PATH'] = '/opt/ros/kinetic/lib/pkgconfig'
ROS_ENVs['CMAKE_PREFIX_PATH'] = '/opt/ros/kinetic'  

## Setup
### roscore
Let's start by making sure that we have roscore running:  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run roscore:
```
$ roscore
```

### turtlesim
For this tutorial we will also use turtlesim.  
open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtlesim_node
```

### turtle keyboard teleoperation
We'll also need something to drive the turtle around with.  

open a new terminal window, run a ROS Docker container with:
```
docker run -it --rm \
	--net=host \
	-v /tmp/.X11-unix:/tmp/.X11-unix \
	-e DISPLAY=unix$DISPLAY \
	--cap-add=SYS_ADMIN \
	--device /dev/snd \
	--device /dev/dri \
	-v /dev/shm:/dev/shm \
	-p 11311:11311 \
	wei1234c/ros_ubuntu_amd64
```

in the container, run:
```
$ rosrun turtlesim turtle_teleop_key
```

Now you can use the arrow keys of the keyboard to drive the turtle around. If you can not drive the turtle, **select the terminal window of the turtle_teleop_key** to make sure that the keys that you type are recorded.

## ROS Services
Services are another way that nodes can communicate with each other. Services allow nodes to send a **request** and receive a **response**.

## Using rosservice
rosservice can easily attach to ROS's client/service framework with services. rosservice has many commands that can be used on topics, as shown below:

Usage:
```
rosservice list         print information about active services
rosservice call         call the service with the provided args
rosservice type         print service type
rosservice find         find services by service type
rosservice uri          print service ROSRPC uri
rosservice list
```

### rosservice list
The list command shows us that the turtlesim node provides nine services: reset, clear, spawn, kill, turtle1/set_pen, /turtle1/teleport_absolute, /turtle1/teleport_relative, turtlesim/get_loggers, and turtlesim/set_logger_level. There are also two services related to the separate rosout node: /rosout/get_loggers and /rosout/set_logger_level.

In [4]:
!rosservice list

/clear
/kill
/reset
/rosout/get_loggers
/rosout/set_logger_level
/spawn
/turtle1/set_pen
/turtle1/teleport_absolute
/turtle1/teleport_relative
/turtlesim/get_loggers
/turtlesim/set_logger_level


### rosservice type
Usage:
```
rosservice type [service]
```
Let's find out what type the clear service is:

In [5]:
!rosservice type /clear

std_srvs/Empty


This service is empty, this means when the service call is made it takes no arguments (i.e. it sends no data when making a request and receives no data when receiving a response). Let's call this service using rosservice call:

### rosservice call
Usage:
```
rosservice call [service] [args]
```
Here we'll call with no arguments because the service is of type empty:

 it clears the background of the turtlesim_node.

In [8]:
!rosservice call /clear

Let's look at the case where the service has arguments by looking at the information for the service spawn:

In [10]:
!rosservice type /spawn

turtlesim/Spawn


In [11]:
!rosservice type /spawn | rossrv show

float32 x
float32 y
float32 theta
string name
---
string name



This service lets us spawn a new turtle at a given location and orientation. The name field is optional, so let's not give our new turtle a name and let turtlesim create one for us.  

The service call returns with the name of the newly created turtle

In [15]:
!rosservice call /spawn 2 2 0.2 ""

name: "turtle2"


## Using rosparam
rosparam allows you to store and manipulate data on the ROS **Parameter Server**. The Parameter Server can store integers, floats, boolean, dictionaries, and lists. rosparam uses the YAML markup language for syntax. In simple cases, YAML looks very natural: 1 is an integer, 1.0 is a float, one is a string, true is a boolean, [1, 2, 3] is a list of integers, and {a: b, c: d} is a dictionary. rosparam has many commands that can be used on parameters, as shown below:

Usage:
```
rosparam set            set parameter
rosparam get            get parameter
rosparam load           load parameters from file
rosparam dump           dump parameters to file
rosparam delete         delete parameter
rosparam list           list parameter names
Let's look at what parameters are currently on the param server:
```
### rosparam list  
Here we can see that the turtlesim node has three parameters on the param server for background color:

In [16]:
!rosparam list

/background_b
/background_g
/background_r
/rosdistro
/roslaunch/uris/host_wei_lenovo_legion__41549
/rosversion
/run_id


### rosparam set and rosparam get
Usage:
```
rosparam set [param_name]
rosparam get [param_name]
```

In [17]:
!rosparam get /background_r

69


In [18]:
!rosparam set /background_r 150

This changes the parameter value, now we have to call the clear service for the parameter change to take effect:

In [19]:
!rosservice call /clear

Now let's look at the values of other parameters on the param server. Let's get the value of the green background channel:

In [21]:
!rosparam get /background_g

86


We can also use rosparam get / to show us the contents of the entire Parameter Server.

In [22]:
!rosparam get /

background_b: 255
background_g: 86
background_r: 150
rosdistro: 'kinetic

  '
roslaunch:
  uris: {host_wei_lenovo_legion__41549: 'http://wei-Lenovo-Legion:41549/'}
rosversion: '1.12.12

  '
run_id: a01a9306-233f-11e8-b375-0242d121c5a4



You may wish to store this in a file so that you can reload it at another time. This is easy using rosparam:

### rosparam dump and rosparam load
Usage:
```
rosparam dump [file_name] [namespace]
rosparam load [file_name] [namespace]
```
Here we write all the parameters to the file params.yaml

In [23]:
!rosparam dump params.yaml

In [24]:
cat params.yaml

background_b: 255
background_g: 86
background_r: 150
rosdistro: 'kinetic

  '
roslaunch:
  uris: {host_wei_lenovo_legion__41549: 'http://wei-Lenovo-Legion:41549/'}
rosversion: '1.12.12

  '
run_id: a01a9306-233f-11e8-b375-0242d121c5a4


You can even load these yaml files into new namespaces, e.g. copy:

In [25]:
!rosparam load params.yaml copy

In [27]:
!rosparam get /copy

background_b: 255
background_g: 86
background_r: 150
rosdistro: 'kinetic

  '
roslaunch:
  uris: {host_wei_lenovo_legion__41549: 'http://wei-Lenovo-Legion:41549/'}
rosversion: '1.12.12

  '
run_id: a01a9306-233f-11e8-b375-0242d121c5a4



In [28]:
!rosparam get /

background_b: 255
background_g: 86
background_r: 150
copy:
  background_b: 255
  background_g: 86
  background_r: 150
  rosdistro: 'kinetic

    '
  roslaunch:
    uris: {host_wei_lenovo_legion__41549: 'http://wei-Lenovo-Legion:41549/'}
  rosversion: '1.12.12

    '
  run_id: a01a9306-233f-11e8-b375-0242d121c5a4
rosdistro: 'kinetic

  '
roslaunch:
  uris: {host_wei_lenovo_legion__41549: 'http://wei-Lenovo-Legion:41549/'}
rosversion: '1.12.12

  '
run_id: a01a9306-233f-11e8-b375-0242d121c5a4

